In [ ]:
!pip install -q keras
!pip install tensorflow

In [36]:
# Importing libraries
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as tfl
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from copy import deepcopy

from sklearn.model_selection import train_test_split
import keras

In [37]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Flatten, Conv1D, MaxPooling1D, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.models import load_model
from keras import backend as K
import csv
import cv2
import torch
import torch.nn as nn

In [38]:
data_train = pd.read_csv('./tabular-playground-series-aug-2022/train.csv')

In [39]:
def clean(data):
  
  le = LabelEncoder()
  
  # Replace str by numbers
  data = deepcopy(data)
  cols = [f"attribute_{i}" for i in range(0, 2)] + ["product_code"]
  for col in cols:
      data[col] = le.fit_transform(data[col])
      #print(le.classes_)
  
  data = data.drop(['id', 'product_code'], axis=1)
  
  # None values
  imputer = SimpleImputer(strategy='mean')
  final_data = pd.DataFrame(imputer.fit_transform(data))

  final_data.columns = data.columns

  return final_data
  
train = clean(data_train)

In [40]:
X = train.drop('failure', axis=1).to_numpy()
Y = train['failure'].astype(np.int32)

train_X, val_X, train_y, val_y = train_test_split(X, Y, test_size=0.2, shuffle=False)
train_y = train_y.to_numpy()
val_y = val_y.to_numpy()

val_y = val_y.reshape(val_y.shape[0], 1)
train_y = train_y.reshape(train_y.shape[0], 1)

In [41]:
train_X[3].astype(np.int32)

array([101,   1,   2,   9,   5,  13,   2,   6,  17,  11,  18,  18,  12,
        19,  12,  16,  18,  10,  15,  15,  16,  17, 826], dtype=int32)

In [42]:
inn = Input((23, 1))
out = inn
out = Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling1D(pool_size=2)(out)
out = Dropout(0.2)(out)
out = Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling1D(pool_size=2)(out)
out = Dropout(0.2)(out)
out = Conv1D(filters=128, kernel_size=3, padding='same', activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling1D(pool_size=2)(out)
out = Dropout(0.2)(out)
out = Conv1D(filters=256, kernel_size=3, padding='same', activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling1D(pool_size=2)(out)
out = Flatten()(out)
out = Dropout(0.2)(out)
# out_shortcut = Dense(units=23, activation='relu', kernel_initializer='he_uniform')(out)
# out = Dropout(0.2)(out)
# out = Dense(units=32, activation='relu', kernel_initializer='he_uniform')(out)
out = Dense(units=1, activation='sigmoid')(out)
model = keras.models.Model(inputs=inn, outputs=out)
model.compile(optimizer='Adam', loss=tf.keras.losses.binary_crossentropy, metrics=["accuracy"])

In [44]:
filepath = "./tabular-playground-series-aug-2022/model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model.fit(train_X, train_y, validation_data=(val_X, val_y), epochs=70, batch_size=64, callbacks = callbacks_list, verbose=2)

Epoch 1/50

Epoch 1: val_accuracy improved from -inf to 0.79281, saving model to ./tabular-playground-series-aug-2022/model.h5
333/333 - 3s - loss: 0.5121 - accuracy: 0.7859 - val_loss: 0.5070 - val_accuracy: 0.7928 - 3s/epoch - 8ms/step
Epoch 2/50

Epoch 2: val_accuracy improved from 0.79281 to 0.79300, saving model to ./tabular-playground-series-aug-2022/model.h5
333/333 - 2s - loss: 0.5125 - accuracy: 0.7860 - val_loss: 0.5043 - val_accuracy: 0.7930 - 2s/epoch - 7ms/step
Epoch 3/50

Epoch 3: val_accuracy did not improve from 0.79300
333/333 - 2s - loss: 0.5119 - accuracy: 0.7858 - val_loss: 0.5037 - val_accuracy: 0.7928 - 2s/epoch - 7ms/step
Epoch 4/50

Epoch 4: val_accuracy did not improve from 0.79300
333/333 - 2s - loss: 0.5126 - accuracy: 0.7859 - val_loss: 0.5033 - val_accuracy: 0.7928 - 2s/epoch - 7ms/step
Epoch 5/50

Epoch 5: val_accuracy did not improve from 0.79300
333/333 - 2s - loss: 0.5127 - accuracy: 0.7859 - val_loss: 0.5036 - val_accuracy: 0.7930 - 2s/epoch - 7ms/step